In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import sum as sum_
from azure.datalake.store import core, lib, multithread
import pandas as pd
from datetime import timedelta

spark = SparkSession.builder \
            .master("local[8]") \
            .appName("airflow_app") \
            .config('spark.executor.memory', '16g') \
            .config('spark.driver.memory', '16g') \
            .config('spark.sql.execution.pandas.respectSessionTimeZone', False) \
            .config("spark.driver.maxResultSize", "2048MB") \
            .config("spark.port.maxRetries", "100") \
            .config("spark.sql.execution.arrow.enabled", "true") \
            .getOrCreate()

adlCreds = lib.auth(url_suffix='raizenprd01', resource='https://datalake.azure.net/')

spark.conf.set("fs.adl.oauth2.access.token.provider.type", "RefreshToken")
spark.conf.set("fs.adl.oauth2.client.id", adlCreds.token['client'])
spark.conf.set("fs.adl.oauth2.refresh.token", adlCreds.token['refreshToken'])

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code B926T5V7R to authenticate.


In [2]:
df = pd.DataFrame()

In [6]:
# Lê dados de predição
df = pd.DataFrame()
output_path = 'adl://raizenprd01.azuredatalakestore.net/ldt_dev/sandbox/dieffgg/previsao_demanda/'
files = ['03_primary/vendas_dia/', '03_primary/SKU/', 
         '03_primary/de_para_IBM_cidades/']
for f in files:
    tmp = spark.read.parquet(output_path + f).toPandas()
    df = pd.concat([df, tmp])
    del tmp
    
# Lê dados de realizado
#realizado = spark.read.format("csv").option("header", "true").load(output_path + './../realizado/realizado.csv').toPandas()

/opt/spark/python/pyspark/sql/dataframe.py:2102: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.8.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40647)
Traceback (most recent call last):
  File "/opt/spark/python/pyspark/sql/dataframe.py", line 533, in collect
    sock_info = self._jdf.collectToPython()
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/opt/spark/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py", line 336, in get_return_value
    format(target_id, ".", name))
py4j.protocol.Py4JError: An error occurred while calling o72.collectToPython

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During hand

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:40647)

In [ ]:
df
